In [1]:
import os
import pickle
import warnings
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasRegressor
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.losses import mean_squared_error
from sklearn.model_selection import train_test_split
from tools import calculate_nse, calculate_rmse, calculate_mae, calculate_mape, calculate_si,calculate_mse
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
warnings.filterwarnings("ignore")
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2023-11-27 07:24:50.847998: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-27 07:24:50.889940: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 07:24:50.889977: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 07:24:50.890007: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 07:24:50.918534: I tensorflow/core/platform/cpu_feature_g

In [2]:
# 读取数据集
datalabel210103 = pd.read_excel('fakoulabeldata/202101-03.xls')
datalabel210406 = pd.read_excel('fakoulabeldata/202104-06.xls')
datalabel210709 = pd.read_excel('fakoulabeldata/202107-09.xls')
datalabel220103 = pd.read_excel('fakoulabeldata/202201-03.xls')
datalabel220406 = pd.read_excel('fakoulabeldata/202204-06.xls')
datalabel220709 = pd.read_excel('fakoulabeldata/202207-09.xls')
datalabel221012 = pd.read_excel('fakoulabeldata/202210-12.xls')
datalabel230103 = pd.read_excel('fakoulabeldata/202301-03.xls')
datalabel230406 = pd.read_excel('fakoulabeldata/202304-06.xls')
datalabel230709 = pd.read_excel('fakoulabeldata/202307-09.xls')

frame = (datalabel210103.iloc[:,:65333],datalabel210406.iloc[:,:65529],datalabel210709.iloc[:,:65530],
       datalabel220103.iloc[:,:65531],datalabel220406.iloc[:,:65525],datalabel220709.iloc[:,:65517],
       datalabel221012.iloc[:,:65522],datalabel230103.iloc[:,:65503],datalabel230406.iloc[:,:65495],
       datalabel230709.iloc[:,:65534])

all_data = pd.concat(frame, ignore_index=True)
selected_data = all_data.iloc[:,1]
selected_data = np.r_[selected_data].reshape(-1, 1)

lookback=8

# 归一化数据
scaler = MinMaxScaler(feature_range=(0, 1))
selected_data = scaler.fit_transform(selected_data)

# 将数据转换为适用于LSTM的输入格式
def create_dataset(dataset):
    X, y = [], []
    for i in range(len(dataset) - lookback):
        X.append(dataset[i:i+lookback])
        y.append(dataset[i+lookback])
    return np.array(X), np.array(y).reshape(-1, 1)

# 将训练集转换为LSTM的输入格式
X, y = create_dataset(selected_data)
X = np.reshape(X, (X.shape[0], -1))

train_size = int(len(X)*0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

np.random.seed(2023)
shuffle_indices = np.random.permutation(len(X_train))
X_train = X_train[shuffle_indices]
y_train = y_train[shuffle_indices]

In [3]:
# 定义LSTM模型
def create_lstm_model(lookback):
    model = Sequential()
    model.add(LSTM(units=32, input_shape=(lookback, 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

lstm_model = KerasRegressor(build_fn=create_lstm_model(lookback=lookback), epochs=20, batch_size=48, verbose=0)
adaboost_lstm = AdaBoostRegressor(base_estimator=lstm_model, n_estimators=2)
adaboost_lstm.fit(X_train, y_train)


2023-11-27 07:25:03.284463: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18859 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:20:00.0, compute capability: 8.6


INFO:tensorflow:Assets written to: ram://24d0b596eadc45189554c233aa3a2e99/assets


INFO:tensorflow:Assets written to: ram://24d0b596eadc45189554c233aa3a2e99/assets
2023-11-27 07:25:07.111302: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://613bb89b668e436bbcf43139455045c2: INVALID_ARGUMENT: ram://613bb89b668e436bbcf43139455045c2 is a directory.
2023-11-27 07:25:09.785937: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8905
2023-11-27 07:25:10.524785: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f195011c100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-27 07:25:10.524823: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-11-27 07:25:10.531657: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-27 07:25:10.684293: I ./tensorflow/compiler/jit/device

INFO:tensorflow:Assets written to: ram://883f127a33f44dd6b2036fa74e173ec2/assets


INFO:tensorflow:Assets written to: ram://883f127a33f44dd6b2036fa74e173ec2/assets
2023-11-27 07:46:35.191392: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://62195871c659469db6519ac0c09301a4: INVALID_ARGUMENT: ram://62195871c659469db6519ac0c09301a4 is a directory.


AdaBoostRegressor(base_estimator=KerasRegressor(batch_size=48, build_fn=<keras.src.engine.sequential.Sequential object at 0x7f1c59d5a350>, epochs=20, verbose=0),
                  n_estimators=2)

In [4]:
predictions = adaboost_lstm.predict(X_test)

# 反归一化预测结果
predictions = scaler.inverse_transform(predictions.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# 计算评价指标
nse = calculate_nse(y_test[:,0], predictions[:,0])
mae = calculate_mae(y_test[:,0], predictions[:,0])
rmse = calculate_rmse(y_test[:,0], predictions[:,0])
mse = calculate_mse(y_test[:,0], predictions[:,0])
si = calculate_si(y_test[:,0], predictions[:,0])
mape = calculate_mape(y_test[:,0], predictions[:,0])

# 输出结果
print("NSE:", nse)
print("MAE:", mae)
print("RMSE:", rmse)
print("MSE:", mse)
print("SI:", si)
print("MAPE:", mape)
# print()


NSE: 0.9998347802530155
MAE: 23.481014436628932
RMSE: 41.07966098566677
MSE: 1687.5385466973128
SI: 0.00835465940357251
MAPE: 1.073791961563408


In [5]:
test_datalabel = pd.read_csv('selected_data.csv').iloc[:,0]
test_datalabel = np.r_[test_datalabel].reshape(-1, 1)
test_scaler = MinMaxScaler(feature_range=(0, 1))
test_data = test_scaler.fit_transform(test_datalabel)
test_dataX, test_datay = create_dataset(test_data)
test_dataX = np.reshape(test_dataX, (test_dataX.shape[0], -1))
test_predictions = adaboost_lstm.predict(test_dataX)

# 反归一化预测结果
test_predictions = test_scaler.inverse_transform(test_predictions.reshape(-1, 1))
test_datay = test_scaler.inverse_transform(test_datay.reshape(-1, 1))

# 计算评价指标
nse = calculate_nse(test_datay[:,0], test_predictions[:,0])
mae = calculate_mae(test_datay[:,0], test_predictions[:,0])
rmse = calculate_rmse(test_datay[:,0], test_predictions[:,0])
mse = calculate_mse(test_datay[:,0], test_predictions[:,0])
si = calculate_si(test_datay[:,0], test_predictions[:,0])
mape = calculate_mape(test_datay[:,0], test_predictions[:,0])

# 输出结果
print("NSE:", nse)
print("MAE:", mae)
print("RMSE:", rmse)
print("MSE:", mse)
print("SI:", si)
print("MAPE:", mape)

NSE: 0.9997929514660394
MAE: 21.863407074002097
RMSE: 45.42376989822924
MSE: 2063.318871767277
SI: 0.007857675151067186
MAPE: 1.0962021873548322
